# 一回でいい

In [14]:
# import
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
import random
from ipywidgets import interact, interact_manual
import ipywidgets as widgets
from IPython import display
import time
from pathlib import Path
import pandas as pd
import os
import copy
import networkx as nx
import pickle
import sys
from collections import Counter
from scipy import stats
%matplotlib inline

# change directory

os.chdir("/home/kyoko/Develop/ActualCausation/MABE/MABE_contents/actual_agency")

# import packages -- pyphi

from pyanimats import *
from pyTPM import *
import actual_agency as agency
import pyphi
from pyphi import actual, config, Direction

pyphi.config.VALIDATE_SUBSYSTEM_STATES = False
pyphi.config.PARTITION_TYPE = 'TRI'

# Change directory back to Example directory, now in MABE_contents
os.chdir("../Experiments/analysis_now")

%load_ext autoreload
%autoreload 2
%aimport pyTPM, pyanimats, actual_agency

# make an array
datas = np.zeros(3000).reshape(250, 12)

prearr = np.sort(np.tile(np.arange(1,11), (1,5))).reshape(50, 1) 
runarr = np.tile(prearr.copy(), (5,1)) #Runs
hidarr = np.sort(np.tile(np.arange(2,7), (1,50))).reshape(250, 1) #Hidden nodes
stsarr = np.tile(np.arange(1,6), (1,50)).reshape(250, 1)

datas[:,0:1] = hidarr
datas[:,1:2] = runarr
datas[:,2:3] = stsarr

# defining some parameters to use 
generations = 30000
n_agents = int(generations/500.+1)
n_runs = 10
n_trials = 64
n_data = 5

agent = 60 # saved generation
trial = 60 # 0~63
run = 1 # 0 ~ n_runs-1 change only this


# Get Line-of-descent (LOD) data from MABE output
    
LOD_data = ['190822_7156331_h2_LOD_data.pkl','190822_7156331_h3_LOD_data.pkl','190822_7156331_h4_LOD_data.pkl','190822_7156331_h5_LOD_data.pkl','190822_7156331_h6_LOD_data.pkl']
LODs = []    
for i in range(len(LOD_data)):
    with open(LOD_data[i],'rb') as f:
        LODs.append(pickle.load(f))

# Get genome data
all_genomes = ['190822_7156331_h2_genome.pkl','190822_7156331_h3_genome.pkl','190822_7156331_h4_genome.pkl','190822_7156331_h5_genome.pkl','190822_7156331_h6_genome.pkl']
genomes = []
for i in range(len(all_genomes)):
    with open(all_genomes[i],'rb') as f:
        genomes.append(pickle.load(f))


# Get activity data
all_activity = ['190822_7156331_h2_activity.pkl','190822_7156331_h3_activity.pkl','190822_7156331_h4_activity.pkl','190822_7156331_h5_activity.pkl','190822_7156331_h6_activity.pkl']
activities = []
for i in range(len(all_activity)):
    with open(all_activity[i],'rb') as f:
        activities.append(pickle.load(f))


# Define functions to find states
# find the most frequent state


def counting3(data):
    c = Counter(data)
    freq_scores = c.most_common()
    if len(freq_scores) < 3:
        return(freq_scores)
    else:
        third_count = freq_scores[2][1]
    
        modes = [freq_scores[0][:], freq_scores[1][:]]
        for p in range(2,len(freq_scores)): #出現回数を比較、追加
            if freq_scores[p][1] == third_count and p < 5:
                modes.append(freq_scores[p])
        return(modes)

# change num(10) to bin(2)
def int2binary(n,nodes):
    holi = [int(x) for x in bin(n)[2:].zfill(nodes)]
    holi.reverse()
    return tuple(holi)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
brain_activity = []
# culculate brain activity the first
for data in range(n_data):
    n_hnodes = data+2
    for n in range(n_runs):
        place = 5*data + n
    
        brain_activity.append(agency.getBrainActivity(activities[data][n],n_agents=n_agents, n_nodes=n_hnodes+4, n_hidden=n_hnodes))
        actarr = brain_activity[place][agent][trial]

        # TPM, TPM_gate, cm
       
        genome.append(agency.get_genome(genomes[data], n, agent))
#print(len(genome[4]))

        a, b, c = genome2TPM(genome[place], n_nodes=n_hnodes+4, n_sensors=2, n_motors=2, gate_type='deterministic',states_convention='loli',remove_sensor_motor_effects=True)
        TPM.append(a)
        TPM_gates.append(b)
        cm.append(c) # right?

Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating a

In [15]:
genome = []
TPM, TPM_gates, cm = [],[],[]
n_connection = np.zeros(n_data*n_runs)

dec = []
label = [['S1','S2','M1','M2','H1','H2'],['S1','S2','M1','M2','H1','H2','H3'],['S1','S2','M1','M2','H1','H2','H3','H4'],['S1','S2','M1','M2','H1','H2','H3','H4','H5'],['S1','S2','M1','M2','H1','H2','H3','H4','H5','H6']]

for data in range(n_data):
    n_hnodes = data+2
    hnodes = [(4,5),(4,5,6),(4,5,6,7),(4,5,6,7,8),(4,5,6,7,8,9)]
    for n in range(n_runs):
        place = 5*data + n
       
        
    # fitness stateなし
    
        LODs[data][n]['fitness'] = (LODs[data][n]['correct_AVE']
                               /(LODs[data][n]['correct_AVE']+LODs[data][n]['incorrect_AVE']))
        #print(LODs[data][n]['fitness'][60])
        
#     # TPM, TPM_gate, cm
       
#         genome.append(agency.get_genome(genomes[data], n, agent))
# #print(len(genome[4]))

#         a, b, c = genome2TPM(genome[place], n_nodes=n_hnodes+4, n_sensors=2, n_motors=2, gate_type='deterministic',states_convention='loli',remove_sensor_motor_effects=True)
#         TPM.append(a)
#         TPM_gates.append(b)
#         cm.append(c) # right?
        
        
        
# culculate num of connection
# for i in range(n_data*n_runs):
#     n_connection[i] = np.count_nonzero(cm[i])

        
        for j in range(len(brain_activity[0][0][0])): # 2進法 -> 10進法変換 行35,列node数の行列 
            num = 0
            for k in range(len(brain_activity[0][0][0][0])): # = hnode + 4
                num = num + (2 ** k) * actarr[j][k] 
                num = int(num) # 2進法への変換のためfloat -> int型
            dec.append(num)

        c = Counter(dec)
        prestate = counting3(dec)
        
        state = []
        for i in range(len(prestate)):
            state.append(int2binary(prestate[i][0],n_hnodes+4))

        # network
        # define network for this animat change
        network = pyphi.network.Network(np.array(TPM), cm=np.array(cm), 
                    node_labels=tuple(label[data]), purview_cache=None)
        # all network of animats; if there is a node not has input and output -> 0

        for i in range(len(state)):
            subsystem = pyphi.Subsystem(network, state[i], nodes=hnodes[data])
            Phi = pyphi.compute.phi(subsystem)
            print("Data %d, Run %d, state %d's phi is %d"%(data, n, i, Phi))


ValueError: Invalid TPM: Must be either 2-dimensional or multidimensional. See the documentation on TPM conventions and the `pyphi.Network` object for more information on TPM forms.

In [ ]:

n_connection = np.zeros(n_data*n_runs)
for i in range(50):
    n_connection[i] = np.count_nonzero(cm[i])
n_connection

In [9]:
prestate

[(36, 11), (16, 8), (4, 8)]

In [13]:
c.most_common()

[(4, 34), (0, 1)]